---
***MNIST ("Modified National Institute of Standards and Technology")*** Handwritten Digit dataset is the de facto “hello world” dataset of computer vision.  
  
Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. A new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike. In this competition, the goal is to correctly identify digits from a dataset of tens of thousands of handwritten images. We will be experimenting with different algorithms to learn first-hand what works well and how techniques compare.

---
# 1) ***Initialization and preparation***
---
## 1.1 Import necessary modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from keras.utils.vis_utils import plot_model
from catboost import CatBoostClassifier
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

## 1.2 Read MNIST data set

In [2]:
df = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

## 1.3 Check for digits with missing labels

In [3]:
print("Missing values in train dataset: {}".format(df.isnull().sum().sum()))
print("Missing values in test dataset: {}".format(test.isnull().sum().sum()))

There is no missing values in the train and test dataset.

---
# 2) Data visualization
---
## 2.1 Target distribution

In [4]:
print("\n")
print(df.label.value_counts())
print("\n")
sns.countplot(df.label)
plt.title("Distribution of handwritten digits")
plt.show()

## 2.2 Check out data columns (image pixel size) and labels

In [5]:
print("Train data set columns: {}".format(df.columns))
print("Test data set columns: {}".format(test.columns))
print("No of pixels: {}".format(len(test.columns)))

## 2.3 Visualize pixels as images

In [6]:
fig, ax = plt.subplots(nrows=1, ncols=4)
for i in range(4):
    ax[i].set_title("Number {}".format(df.loc[i][0]))
    ax[i].imshow(np.array(df.loc[i][1:]).reshape(28,28),cmap='gray')
    ax[i].axis('off')
fig.tight_layout(pad=0.0)
plt.show()

---
# 3.0 Modelling
---

## 3.1 Train-development split of data
We first split the dataset into training and developmental data set with train_test_split.

In [7]:
train, dev = train_test_split(df,test_size = 0.2, random_state = 42)

# Baseline Model
We incorporate some baseline models which are less complex to see how much we can do in as little time as possible, then optimize from there upwards.  
We will be using:
* Logistic Regression (Multinomial)
* Cat Gradient Boosting Classifier
* K Nearest Neighbors
* Support Vector Classifier

In [8]:
logreg = LogisticRegression(multi_class='multinomial')
logreg.fit(train.drop('label',axis=1),train.label)
logreg_pred = logreg.predict(dev.drop('label',axis=1))
print("Accuracy : {}".format(round(accuracy_score(dev.label,logreg_pred),4)))

In [9]:
cat = CatBoostClassifier(loss_function='MultiClass',n_estimators = 100,verbose = 0)
cat.fit(train.drop('label',axis=1),train.label, eval_set=(dev.drop('label',axis=1), dev.label))
cat_pred = cat.predict(dev.drop('label',axis=1))
print("Accuracy : {}".format(round(accuracy_score(dev.label,cat_pred),4)))

In [10]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(train.drop('label',axis=1),train.label)
knn_pred = knn.predict(dev.drop('label',axis=1))
print("Accuracy : {}".format(round(accuracy_score(dev.label,knn_pred),4)))

In [11]:
svc = SVC()
svc.fit(train.drop('label',axis=1),train.label)
svc_pred = svc.predict(dev.drop('label',axis=1))
print("Accuracy : {}".format(round(accuracy_score(dev.label,svc_pred),4)))

We can see that our baseline models can already do a pretty good work with > 97% accuracy to predict the digits. Can we do better with a more complex model?

---
# Convolutional Neural Network
---
This will be our main model used to predict the test data set. The reason why we picked it as our final model is because CNN has had a great reputation with dealing with images especially large images. There are several reasons why and how:  
  
* CNN is particularly useful as the ***convolutional layer*** can be used to automatically extract important features from the images and gradually build up to a higher-level more important features as the layers get deeper. We chose to use ***same padding*** to pad around image edges during the convolutional process as the original image size is already small enough (28 x 28 pixels) and we can't afford to lose too many important data.  

* We also add in ***MaxPooling*** into our convolutional neural network (though only 2 times) although it can downsize the image sizes but it does increase training speed while it subsamples the image. At the same time, MaxPooling preserve the most important feature while downsizing and offers a benefit of translation invariance (the network learns to desensitize itself against fixed location so that it doesn't remember where the pixel location starts.)  
  
* ***BatchNormalization*** is also introduced within the layers to allow every layer of the network to do learning more independently. It is used to normalize the output of neurons and makes learning faster.  

* For regularization, other than BatchNormaliztion, we also tried out using ***Spatial Dropout***. It performs simillar function as Dropout, however, it drops entire 2D feature maps instead of individual elements. If adjacent pixels within feature maps are strongly correlated (as is normally the case in early convolution layers) then regular dropout will not regularize the activations and will otherwise just result in an effective learning rate decrease. In this case, SpatialDropout2D will help promote independence between feature maps and should be used instead.
  
* For the activation layer, we used the famous ***ReLu activation function*** as ReLu is empirically found to work well. Papers have observed that training a deep network with ReLu tended to converge much more quickly and reliably than training a deep network with a normal sigmoid activation. It also introduces non-linearity into the model at the same time.  
  
* Finally, we ***flatten*** the image as feature columns and pass it into a neural network with ***two hidden layers*** and one ***output layer with softmax*** to denote the probability distributions for each target.

* Note that we also added two data augmentation layer infront of the model to increase our data sizes. We did ***random rotation*** of image up to 18% and a ***random zoom*** of height or width within 20%.

The full model plot can be seen below after defining the model.

While fitting the model, we include ***early stopping*** to prevent our model from overfitting to the training set due to its complexity.  
Other than that, we try to squeeze every possible juice using ***Reduce Learning Rate on Plateau*** as models often benefit from reducing the learning rate when learning stagnates.

V2.0: To further compete on the leaderboard, we further optimize the model's ability to generalize by using an ensemble of 5 CNN (though the same layers). We sum up the probability distributions before we perform argmax over the distribution (so it is somewhat similar to voting, but more accurate). Using the same model but different fitting process will often end up with different results, this is due to stochastic nature of random initializing the weights of the neurons.

---
But first, we will divide the values of pixels within the train, development and test sets by 255 to normalize the pixels.  
After that we will reshape them so that they can fit into the input layer of the Keras API.

In [12]:
train_X = train.drop('label',axis=1).values
train_X = train_X / 255
train_X = train_X.reshape(-1,28,28,1)

dev_X = dev.drop('label',axis=1).values
dev_X = dev_X / 255
dev_X = dev_X.reshape(-1,28,28,1)

train_y = train.label.values
train_y = train_y.reshape(-1,1)

dev_y = dev.label.values
dev_y = dev_y.reshape(-1,1)

test = test.values / 255
test = test.reshape(-1,28,28,1)

In [13]:
model = models.Sequential()
model.add(layers.RandomRotation(factor=0.05, fill_mode='constant'))  
model.add(layers.RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2), fill_mode='constant'))

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same',strides = 1, input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding = 'same',strides = 1))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2))

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same',strides = 1))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'same',strides = 1))
model.add(layers.MaxPooling2D(2))
model.add(layers.SpatialDropout2D(0.4))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'same',strides = 1))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'same',strides = 1))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same',strides = 1))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same',strides = 1))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())

model.add(layers.Dense(52, activation='relu'))
model.add(layers.Dropout(0.4))

model.add(layers.Dense(26, activation='relu'))
model.add(layers.Dropout(0.4))

model.add(layers.Dense(10,activation = 'softmax'))

In [15]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    min_delta = 0.0001,
    restore_best_weights=True,
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    min_lr=0.00001,
    factor=0.7,
    patience=5
)

In [16]:
model_count = 5
history  = [0] * model_count
results = [0] * model_count
dev_results = [0] * model_count


for i in range(model_count):
  print("Model {}".format(i + 1))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
  history[i] = model.fit(train_X,train_y,
      validation_data=(dev_X,dev_y),
      batch_size=64,
      callbacks=[early_stopping, reduce_lr],
      epochs=100,
      verbose=1)

  results[i] = model.predict(test)
  dev_results[i] = model.predict(dev_X)

In [22]:
plot_model(model, show_shapes=True, show_layer_names=True)

As we mentioned, the final results were taken by using the argmax of the sum of five different probability distributions from five different networks.

In [17]:
ensemble_cnn_pred = np.argmax(sum(results),axis = 1)
ensemble_cnn_dev = np.argmax(sum(dev_results),axis = 1)

---
# 4.0 Results Visualization
---
Here, we plot and observe one of the training history among the five networks.  
We also plot the confusion matrix of the developmental data to see which are the most frequent incorrect numbers.  
(We could work on from there by adding more of such data, but due to the nature of the competition, I did not add in more additional data.)

In [18]:
# Plot the loss and accuracy curves for one of the networks 
fig, ax = plt.subplots(2,1, figsize=(18, 10))
ax[0].plot(history[0].history['loss'], color='b', label="Training loss")
ax[0].plot(history[0].history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history[0].history['sparse_categorical_accuracy'], color='b', label="Training accuracy")
ax[1].plot(history[0].history['val_sparse_categorical_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [19]:
if_cm = confusion_matrix(dev_y, ensemble_cnn_dev)
sns.heatmap(if_cm, annot=True,annot_kws={"size": 16},fmt='g')
plt.title("Ensemble CNN Confusion Matrix")
plt.show()

In [20]:
fig, ax = plt.subplots(nrows=1, ncols=4)
for i in range(4):
    ax[i].set_title("Number {}".format(ensemble_cnn_pred[i]))
    ax[i].imshow(np.array(test[i][:]).reshape(28,28),cmap='gray')
    ax[i].axis('off')
fig.tight_layout(pad=0.0)
plt.show()

---
# 5.0 Submission
---
The models achieved a combined accuracy of 99.546% which places me around Top 7 ~ 8 % in the leaderboard as of writing.

In [21]:
submission=pd.DataFrame(zip(list(range(1,len(ensemble_cnn_pred)+1)),ensemble_cnn_pred),columns=['ImageId','Label'])
submission.to_csv('submission.csv',index = False)

# Thanks!
If you have any questions, comments or suggestions feel free to comment. It would be much appreciated!